### Star Ratings Results
- Input from AimBig notebook, stat_regression 
- 6 steps of the star rating results
- GroupBy object = (Contract ID, ESA Code, Speciality_site_type_code)

In [34]:
import datetime as dt
import re
import json
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from datetime import date, timedelta
from datetime import datetime
import warnings
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sn
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

# reading in file with numerator/denominator for each participant, performance measure against contract
df_merged = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_merge_final.csv')
# creating groupBy object 
df_merged = df_merged.drop_duplicates()
#df_group = df_merged.groupby(['CONTRACT_ID', 'ESA_CODE', 'SPECIALIST_SITE_TYPE_CODE'])

df_merged.columns

df_merged = df_merged.drop(columns=['Unnamed: 0'])

Columns missing due to no outcomes = 'Num_26_path', 'Num_26_wrkast', 'Den_26_path', 'Den_26_wrkast', 'Den_52_path', 'Num_52_path'
- set as 0 

In [35]:
df_merged

,Program,JOB_SEEKER_ID,CONTRACT_ID,SITE_CODE,SITE_DESCRIPTION,ESA_CODE,ESA Name,State,SPECIALIST_SITE_TYPE_CODE,Num_13_full,...,Num_26_path_exp,Num_26_wrkast,Den_26_wrkast,Num_26_wrkast_exp,Num_52_full,Den_52_full,Num_52_full_exp,Num_52_path,Den_52_path,Num_52_path_exp
0,ESS,168470,0212898K,AB31,AimBig Employment AUBURN,4CWS,Northern Sydney,NSW,AALL,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,DMS,451780,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,QLD,MUSK,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,DMS,3606070,0212846K,MM07,AimBig Employment BURWOOD,4INW,South East Sydney,NSW,MENH,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,DMS,618010,0212823C,MN07,AimBig Employment SUNSHINE,4WES,Western Victoria,VIC,MUSK,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DMS,630040,0212831C,MM67,AimBig Employment ROCKINGHAM,4CWM,Western Australia,WA,MENH,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5307,DMS,2730016019,0212841E,Y947,AimBig Employment MERRYLANDS,4CWS,Northern Sydney,NSW,AALL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5308,DMS,2696040,0212857B,ML77,AimBig Employment WOLLONGONG,4WOL,South East Sydney,NSW,MUSK,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5309,DMS,5892574007,0212825E,MM77,AimBig Employment MELBOURNE,4YAR,Northern Victoria,VIC,MUSK,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5310,DMS,832617003,0212823C,YA97,AimBig Employment Pty Ltd ST ALBANS,4WES,Western Victoria,VIC,MUSK,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
print(df_merged.isna().sum())

Program                         0
JOB_SEEKER_ID                   0
CONTRACT_ID                     0
SITE_CODE                       0
SITE_DESCRIPTION                0
ESA_CODE                        0
ESA Name                        0
State                           0
SPECIALIST_SITE_TYPE_CODE       0
Num_13_full                  3813
Den_13_full                  3822
Num_13_exp                   3813
Num_26_full                  3813
Den_26_full                  3813
Num_26_full_exp              3813
Num_26_path                  3813
Den_26_path                  3813
Num_26_path_exp              3813
Num_26_wrkast                3813
Den_26_wrkast                3813
Num_26_wrkast_exp            3813
Num_52_full                  3813
Den_52_full                  3813
Num_52_full_exp              3813
Num_52_path                  3813
Den_52_path                  3813
Num_52_path_exp              3813
dtype: int64


### Step 1) Calculating Actual Performance
#### Actual Performance
groupBy key = (Contract ID, ESA Code, Speciality_site_type_code)

##### Pre-Quarter

- Count NaNs -> place as 0

In [37]:
df_merged = df_merged.fillna(value = 0)
df_merged

,Program,JOB_SEEKER_ID,CONTRACT_ID,SITE_CODE,SITE_DESCRIPTION,ESA_CODE,ESA Name,State,SPECIALIST_SITE_TYPE_CODE,Num_13_full,...,Num_26_path_exp,Num_26_wrkast,Den_26_wrkast,Num_26_wrkast_exp,Num_52_full,Den_52_full,Num_52_full_exp,Num_52_path,Den_52_path,Num_52_path_exp
0,ESS,168470,0212898K,AB31,AimBig Employment AUBURN,4CWS,Northern Sydney,NSW,AALL,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,DMS,451780,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,QLD,MUSK,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,DMS,3606070,0212846K,MM07,AimBig Employment BURWOOD,4INW,South East Sydney,NSW,MENH,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,DMS,618010,0212823C,MN07,AimBig Employment SUNSHINE,4WES,Western Victoria,VIC,MUSK,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DMS,630040,0212831C,MM67,AimBig Employment ROCKINGHAM,4CWM,Western Australia,WA,MENH,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5307,DMS,2730016019,0212841E,Y947,AimBig Employment MERRYLANDS,4CWS,Northern Sydney,NSW,AALL,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5308,DMS,2696040,0212857B,ML77,AimBig Employment WOLLONGONG,4WOL,South East Sydney,NSW,MUSK,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5309,DMS,5892574007,0212825E,MM77,AimBig Employment MELBOURNE,4YAR,Northern Victoria,VIC,MUSK,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5310,DMS,832617003,0212823C,YA97,AimBig Employment Pty Ltd ST ALBANS,4WES,Western Victoria,VIC,MUSK,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
df_merged.columns

Index(['Program', 'JOB_SEEKER_ID', 'CONTRACT_ID', 'SITE_CODE',
       'SITE_DESCRIPTION', 'ESA_CODE', 'ESA Name', 'State',
       'SPECIALIST_SITE_TYPE_CODE', 'Num_13_full', 'Den_13_full', 'Num_13_exp',
       'Num_26_full', 'Den_26_full', 'Num_26_full_exp', 'Num_26_path',
       'Den_26_path', 'Num_26_path_exp', 'Num_26_wrkast', 'Den_26_wrkast',
       'Num_26_wrkast_exp', 'Num_52_full', 'Den_52_full', 'Num_52_full_exp',
       'Num_52_path', 'Den_52_path', 'Num_52_path_exp'],
      dtype='object')

### Aggregating to Site Level 

In [39]:
df_group = df_merged.groupby(['Program', 'CONTRACT_ID', 'SITE_DESCRIPTION', 'ESA_CODE', 'SPECIALIST_SITE_TYPE_CODE'])['Num_13_full', 'Den_13_full', 'Num_13_exp', 'Num_26_full', 'Den_26_full', 'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast', 'Num_52_full', 'Den_52_full', 'Num_52_full_exp', 'Num_52_path', 'Den_52_path', 'Num_52_path_exp'].sum()

df_group.to_csv('df_group_fin.csv')
df_group['actual_13_full'] = df_group['Num_13_full']/df_group['Den_13_full'] * 100
df_group['actual_26_full'] = df_group['Num_26_full']/df_group['Den_26_full'] * 100

df_group['actual_26_path'] = df_group['Num_26_path']/df_group['Den_26_path'] * 100
df_group['actual_26_wrkast'] = df_group['Num_26_wrkast']/df_group['Den_26_wrkast'] * 100
df_group['actual_52_full'] = df_group['Num_52_full']/df_group['Den_52_full'] * 100
df_group['actual_52_path'] = df_group['Num_52_path']/df_group['Den_52_path'] * 100




#### Renaming df_group -> df_pre_actual

In [40]:
df_pre_actual = df_group
df_pre_actual.to_csv('C:/Users/AManalo/star_ratings_new/Output Files/test_sites.csv')

In [41]:
def get_actPerf(type, df_group):
    '''Input:
        - type: 'pre' or 'post' quarter
        - df_group: groupBy object that contains site participants with num/den
        Output:
        - returns dataframe with pre/post quarter actual performance for each Contract'''
    
    if type == 'pre':
        df_group['preAct13_full'] = 0 # initialise pre_actual_rate column
        df_group['preAct26_path'] = 0
        df_group['preAct26_full'] = 0
        df_group['preAct26_wrkast'] = 0
        df_group['preAct52_full'] = 0
        df_group['preAct52_path'] = 0
        

    elif type == 'post':
        df_group['postAct13_full'] = 0 # initialise post_actual_rate column
        df_group['postAct26_path'] = 0
        df_group['postAct26_full'] = 0
        df_group['postAct26_wrkast'] = 0
        df_group['postAct52_full'] = 0
        df_group['postAct52_path'] = 0

    

    return df_group

##### Post-Quarter

In [42]:
# reading in the merged file
df_merge_post = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_merge_post.csv')
df_merge_post = df_merge_post.fillna(value = 0)
df_merge_post.head(5)

,Unnamed: 0,JOB_SEEKER_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,Den_26_wrkast,Num_52_full,Den_52_full,Num_52_path,Den_52_path
0,0,9386301003,0,0,AimBig Employment WOY WOY,0212812K,AALL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,8653691003,0,0,AimBig Employment FAIRFIELD,0212899A,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
2,2,7870290004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,8.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0
3,3,6638331004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2580218019,0,0,AimBig Employment LEICHHARDT,0212900K,AALL,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


This is WITHOUT 52 week outcomes -> ONLY FOR TESTING 13 and 26 week

In [43]:
df_merge_post = df_merge_post.rename(columns={'Num_13' : 'Num_13_full', 'Den_13' : 'Den_13_full'})

In [44]:
df_group = df_merge_post.groupby(['Contract_ID', 'Site_Name', 'specialist_site_type_code', 'JOB_SEEKER_ID'])['Num_13_full', 'Den_13_full', 'Num_26_full', 'Den_26_full', 'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast', 'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].sum()
df_group = df_merge_post.groupby(['Contract_ID', 'Site_Name', 'specialist_site_type_code'])['Num_13_full', 'Den_13_full', 'Num_26_full', 'Den_26_full', 'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast', 'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].sum()


df_group.to_csv('C:/Users/AManalo/star_ratings_new/df_group_fin.csv')

Calculating Actual Performance

In [45]:
# renaming df_13 ->



df_group['actual_13_full'] = df_group['Num_13_full']/df_group['Den_13_full'] * 100
df_group['actual_26_full'] = df_group['Num_26_full']/df_group['Den_26_full'] * 100

df_group['actual_26_path'] = df_group['Num_26_path']/df_group['Den_26_path'] * 100
df_group['actual_26_wrkast'] = df_group['Num_26_wrkast']/df_group['Den_26_wrkast'] * 100
df_group['actual_52_full'] = df_group['Num_52_full']/df_group['Den_52_full'] * 100
df_group['actual_52_path'] = df_group['Num_52_path']/df_group['Den_52_path'] * 100

df_group = df_group.fillna(0) # if numerator is a 0, then division by a 0 results in a NaN -> but by documentation, should be 0.

df_group.to_csv('C:/Users/AManalo/star_ratings_new/df_group_test.csv')

In [46]:
df_group

Num_13_full  \
Contract_ID Site_Name                         specialist_site_type_code                
0212789F    AimBig Employment PROSPECT        (blank)                              0   
0212790K    AimBig Employment PROSPECT        MUSK                                 0   
0212791K    AimBig Employment MILE END        MENH                                 0   
            AimBig Employment Pty Ltd. SEATON MENH                                 0   
0212792A    AimBig Employment MILE END        (blank)                              0   
...                                                                              ...   
0212902B    AimBig Employment INGLEBURN       AALL                                 0   
0212903C    AimBig Employment PENRITH         (blank)                              0   
0212904D    AimBig Employment HURSTVILLE      AALL                                 0   
            AimBig Employment MIRANDA         AALL                                 0   
            AimBig Employment ROCKDALE        AALL                                 0   

                                                                         Den_13_full  \
Contract_ID Site_Name                         specialist_site_type_code                
0212789F    AimBig Employment PROSPECT        (blank)                             22   
0212790K    AimBig Employment PROSPECT        MUSK                                15   
0212791K    AimBig Employment MILE END        MENH                                15   
            AimBig Employment Pty Ltd. SEATON MENH                                 6   
0212792A    AimBig Employment MILE END        (blank)                              4   
...                                                                              ...   
0212902B    AimBig Employment INGLEBURN       AALL                                24   
0212903C    AimBig Employment PENRITH         (blank)                             30   
0212904D    AimBig Employment HURSTVILLE      AALL                                25   
            AimBig Employment MIRANDA         AALL                                27   
            AimBig Employment ROCKDALE        AALL                                12   

                                                                         Num_26_full  \
Contract_ID Site_Name                         specialist_site_type_code                
0212789F    AimBig Employment PROSPECT        (blank)                            1.0   
0212790K    AimBig Employment PROSPECT        MUSK                               1.0   
0212791K    AimBig Employment MILE END        MENH                               1.0   
            AimBig Employment Pty Ltd. SEATON MENH                               0.0   
0212792A    AimBig Employment MILE END        (blank)                            1.0   
...                                                                              ...   
0212902B    AimBig Employment INGLEBURN       AALL                               0.0   
0212903C    AimBig Employment PENRITH         (blank)                            2.0   
0212904D    AimBig Employment HURSTVILLE      AALL                               2.0   
            AimBig Employment MIRANDA         AALL                               0.0   
            AimBig Employment ROCKDALE        AALL                               0.0   

                                                                         Den_26_full  \
Contract_ID Site_Name                         specialist_site_type_code                
0212789F    AimBig Employment PROSPECT        (blank)                           41.0   
0212790K    AimBig Employment PROSPECT        MUSK                              17.0   
0212791K    AimBig Employment MILE END        MENH                              25.0   
            AimBig Employment Pty Ltd. SEATON MENH                               0.0   
0212792A    AimBig Employment MILE END        (blank)                           25.0   
...                   

### Validating Contracts

## Step 2: Expected Performance

Using statistical regression, which takes account of variable Participant and labour market characteristics and the number of Participants in the denominator post-quarterisation, the Star Ratings model calculates the number of outcomes that the Contract could be expected to achieve.

In [47]:
df_merged.columns

Index(['Program', 'JOB_SEEKER_ID', 'CONTRACT_ID', 'SITE_CODE',
       'SITE_DESCRIPTION', 'ESA_CODE', 'ESA Name', 'State',
       'SPECIALIST_SITE_TYPE_CODE', 'Num_13_full', 'Den_13_full', 'Num_13_exp',
       'Num_26_full', 'Den_26_full', 'Num_26_full_exp', 'Num_26_path',
       'Den_26_path', 'Num_26_path_exp', 'Num_26_wrkast', 'Den_26_wrkast',
       'Num_26_wrkast_exp', 'Num_52_full', 'Den_52_full', 'Num_52_full_exp',
       'Num_52_path', 'Den_52_path', 'Num_52_path_exp'],
      dtype='object')

In [48]:
# Get expected columns from df_merged
df_merged.columns
df_exp = df_merged[['JOB_SEEKER_ID', 'Program', 'CONTRACT_ID',  'SITE_DESCRIPTION', 'ESA_CODE','SPECIALIST_SITE_TYPE_CODE', 'Num_13_exp', 'Num_26_full_exp', 'Num_26_path_exp', 'Num_26_wrkast_exp', 'Num_52_full_exp', 'Num_52_path_exp']]
       

Merging df_exp with the denominator from post quarterisation outcomes

In [49]:
df_merge_post.columns

Index(['Unnamed: 0', 'JOB_SEEKER_ID', 'Num_13_full', 'Den_13_full',
       'Site_Name', 'Contract_ID', 'specialist_site_type_code', 'Num_26_full',
       'Den_26_full', 'Num_26_path', 'Den_26_path', 'Num_26_wrkast',
       'Den_26_wrkast', 'Num_52_full', 'Den_52_full', 'Num_52_path',
       'Den_52_path'],
      dtype='object')

In [50]:
df_exp

,JOB_SEEKER_ID,Program,CONTRACT_ID,SITE_DESCRIPTION,ESA_CODE,SPECIALIST_SITE_TYPE_CODE,Num_13_exp,Num_26_full_exp,Num_26_path_exp,Num_26_wrkast_exp,Num_52_full_exp,Num_52_path_exp
0,168470,ESS,0212898K,AimBig Employment AUBURN,4CWS,AALL,0.0,0.0,0.0,0.0,0.0,0.0
1,451780,DMS,0212875D,AimBig Employment CALOUNDRA,4SUC,MUSK,0.0,0.0,0.0,0.0,0.0,0.0
2,3606070,DMS,0212846K,AimBig Employment BURWOOD,4INW,MENH,0.0,0.0,0.0,0.0,0.0,0.0
3,618010,DMS,0212823C,AimBig Employment SUNSHINE,4WES,MUSK,0.0,0.0,0.0,0.0,0.0,0.0
4,630040,DMS,0212831C,AimBig Employment ROCKINGHAM,4CWM,MENH,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5307,2730016019,DMS,0212841E,AimBig Employment MERRYLANDS,4CWS,AALL,0.0,0.0,0.0,0.0,0.0,0.0
5308,2696040,DMS,0212857B,AimBig Employment WOLLONGONG,4WOL,MUSK,0.0,0.0,0.0,0.0,0.0,0.0
5309,5892574007,DMS,0212825E,AimBig Employment MELBOURNE,4YAR,MUSK,0.0,0.0,0.0,0.0,0.0,0.0
5310,832617003,DMS,0212823C,AimBig Employment Pty Ltd ST ALBANS,4WES,MUSK,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
df_merge_post
df_post_den = df_merge_post[["JOB_SEEKER_ID", "Den_13_full", "Den_26_full", "Den_26_path", "Den_26_wrkast", "Den_52_full", "Den_52_path"]]
df_exp_per = df_exp.merge(df_post_den, on=['JOB_SEEKER_ID'], how='inner')

df_exp_per = df_exp_per.groupby(['Program', 'CONTRACT_ID', 'SITE_DESCRIPTION', 'ESA_CODE', 'SPECIALIST_SITE_TYPE_CODE'])[ "Den_13_full", "Den_26_full", "Den_26_path", "Den_26_wrkast", "Den_52_full", "Den_52_path", 'Num_13_exp', 'Num_26_full_exp', 'Num_52_full_exp', 'Num_52_path_exp', 'Num_26_wrkast_exp', 'Num_26_path_exp'].sum()



Calculating expected performance rate
- for each performance measure

In [52]:

df_exp_per['exPer_13_full'] = df_exp_per['Den_13_full']/df_exp_per['Num_13_exp'] * 100
df_exp_per['exPer_26_full'] = df_exp_per['Den_26_full']/df_exp_per['Num_26_full_exp'] * 100
df_exp_per['exPer_26_path'] = df_exp_per['Den_26_path']/df_exp_per['Num_26_path_exp'] * 100
df_exp_per['exPer_26_wrkast'] = df_exp_per['Den_26_wrkast']/df_exp_per['Num_26_wrkast_exp'] * 100
df_exp_per['exPer_52_full'] = df_exp_per['Den_52_full']/df_exp_per['Num_52_full_exp'] * 100
df_exp_per['exPer_52_path'] = df_exp_per['Den_52_path']/df_exp_per['Num_52_path_exp'] * 100

df_exp_per = df_exp_per.fillna(0)

df_exp_per

Den_13_full  \
Program CONTRACT_ID SITE_DESCRIPTION                  ESA_CODE SPECIALIST_SITE_TYPE_CODE                
DMS     0212789F    AimBig Employment PROSPECT        4EAA     MENH                                 9   
        0212790K    AimBig Employment PROSPECT        4EAA     MUSK                                 3   
        0212791K    AimBig Employment MILE END        4WEA     MENH                                12   
                    AimBig Employment Pty Ltd. SEATON 4WEA     MENH                                 6   
        0212793B    AimBig Employment BURLEIGH HEADS  4GOC     MENH                                12   
...                                                                                               ...   
ESS     0212902B    AimBig Employment INGLEBURN       4MAC     AALL                                10   
        0212903C    AimBig Employment PENRITH         4NEP     AALL                                 4   
        0212904D    AimBig Employment HURSTVILLE      4SGS     AALL                                11   
                    AimBig Employment MIRANDA         4SGS     AALL                                12   
                    AimBig Employment ROCKDALE        4SGS     AALL                                12   

                                                                                          Den_26_full  \
Program CONTRACT_ID SITE_DESCRIPTION                  ESA_CODE SPECIALIST_SITE_TYPE_CODE                
DMS     0212789F    AimBig Employment PROSPECT        4EAA     MENH                              40.0   
        0212790K    AimBig Employment PROSPECT        4EAA     MUSK                               0.0   
        0212791K    AimBig Employment MILE END        4WEA     MENH                              17.0   
                    AimBig Employment Pty Ltd. SEATON 4WEA     MENH                               0.0   
        0212793B    AimBig Employment BURLEIGH HEADS  4GOC     MENH                              16.0   
...                                                                                               ...   
ESS     0212902B    AimBig Employment INGLEBURN       4MAC     AALL                               8.0   
        0212903C    AimBig Employment PENRITH         4NEP     AALL                               8.0   
        0212904D    AimBig Employment HURSTVILLE      4SGS     AALL                              17.0   
                    AimBig Employment MIRANDA         4SGS     AALL                               8.0   
                    AimBig Employment ROCKDALE        4SGS     AALL                               0.0   

                                                                                          Den_26_path  \
Program CONTRACT_ID SITE_DESCRIPTION                  ESA_CODE SPECIALIST_SITE_TYPE_CODE                
DMS     0212789F    AimBig Employment PROSPECT        4EAA     MENH                               5.0   
        0212790K    AimBig Employment PROSPECT        4EAA     MUSK                               0.0   
        0212791K    AimBig Employment MILE END        4WEA     MENH                              10.0   
                    AimBig Employment Pty Ltd. SEATON 4WEA     MENH                               8.0   
        0212793B    AimBig Employment BURLEIGH HEADS  4GOC     MENH                               2.0   
...                                                                                               ...   
ESS     0212902B    AimBig Employment INGLEBURN       4MAC     AALL                               1.0   
        0212903C    AimBig Employment PENRITH         4NEP     AALL                               1.0   
        0212904D    AimBig Employment HURSTVILLE      4SGS     AALL                               2.0   
                    AimBig Employment MIRANDA         4SGS     AALL                               1.0   
                    AimBig Employment ROCKDALE        4SGS     AALL                               

## Step 3: Calculate the Performance Measure Scores (Ratio of Actual Outcomes : Expected Outcomes)

Combined 13, 25, 52 week scores are calculated based on the weightings of each sub type measure. 
Actual rate: actual rate (post-quarter)
Expected rate: expected performance rate


To-Fix: Need to adjust the weights for each performance measure IF the denominator in the pre-quarter outcomes are 0 or very small. 

- ESS: min 20 participants (combined) in the pre-quarterisation denominators for the following: 13-week full outcomes (min 5 participants), ongoing support  
- DMS: min 20 participants in the pre-quarterisation denominator for 13 week full outcomes measure

In [53]:
df_group.reset_index()

,Contract_ID,Site_Name,specialist_site_type_code,Num_13_full,Den_13_full,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,...,Num_52_full,Den_52_full,Num_52_path,Den_52_path,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path
0,0212789F,AimBig Employment PROSPECT,(blank),0,22,1.0,41.0,0.0,5.0,0.0,...,2.0,42.0,0.0,0.0,0.0,2.439024,0.000000,0.0,4.761905,0.0
1,0212790K,AimBig Employment PROSPECT,MUSK,0,15,1.0,17.0,0.0,2.0,0.0,...,0.0,16.0,0.0,0.0,0.0,5.882353,0.000000,0.0,0.000000,0.0
2,0212791K,AimBig Employment MILE END,MENH,0,15,1.0,25.0,1.0,11.0,0.0,...,1.0,25.0,0.0,0.0,0.0,4.000000,9.090909,0.0,4.000000,0.0
3,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,0,6,0.0,0.0,1.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,12.500000,0.0,0.000000,0.0
4,0212792A,AimBig Employment MILE END,(blank),0,4,1.0,25.0,0.0,3.0,0.0,...,0.0,24.0,0.0,0.0,0.0,4.000000,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,0212902B,AimBig Employment INGLEBURN,AALL,0,24,0.0,8.0,0.0,1.0,0.0,...,1.0,9.0,0.0,0.0,0.0,0.000000,0.000000,0.0,11.111111,0.0
216,0212903C,AimBig Employment PENRITH,(blank),0,30,2.0,42.0,0.0,5.0,0.0,...,2.0,34.0,0.0,0.0,0.0,4.761905,0.000000,0.0,5.882353,0.0
217,0212904D,AimBig Employment HURSTVILLE,AALL,0,25,2.0,34.0,0.0,4.0,0.0,...,0.0,32.0,0.0,0.0,0.0,5.882353,0.000000,0.0,0.000000,0.0
218,0212904D,AimBig Employment MIRANDA,AALL,0,27,0.0,16.0,0.0,2.0,0.0,...,0.0,16.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0


In [71]:
df_exp_per.to_csv('C:/Users/AManalo/star_ratings_new/expected_outcomes.csv')

In [55]:
# subsetting the post-quarter actual rates for each contract
df_actPer = df_group.reset_index() # converting groupBy obj -> dataframe for subsetting
df_actPer.columns
df_actPer = df_actPer[['Contract_ID',  'Site_Name',  'specialist_site_type_code', 'actual_13_full',
       'actual_26_full', 'actual_26_path', 'actual_26_wrkast',
       'actual_52_full', 'actual_52_path']]

# subsetting the expected performance for each contract
df_exp_per = df_exp_per.reset_index()
df_exp_per.columns
df_exp_per = df_exp_per[['CONTRACT_ID','SITE_DESCRIPTION', 'SPECIALIST_SITE_TYPE_CODE', 'exPer_13_full', 'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast',
       'exPer_52_full', 'exPer_52_path']]
df_exp_per = df_exp_per.rename(columns={'CONTRACT_ID' : 'Contract_ID', 'SITE_DESCRIPTION' : 'Site_Name', 'SPECIALIST_SITE_TYPE_CODE' : 'specialist_site_type_code'})
# joining the two dataframes
df_perMea = df_actPer.merge(df_exp_per, on=['Contract_ID',  'Site_Name',  'specialist_site_type_code'], how='inner')

df_perMea.columns

df_perMea.isna().sum()
df_perMea.replace([np.inf, -np.inf], 0, inplace=True)

Creating the performance measure scores
### replace weightings - DMS, ESS specific 

In [69]:
df_perMea = df_perMea.groupby(['Contract_ID',  'Site_Name',  'specialist_site_type_code'])['actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path', 'exPer_13_full',
       'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast', 'exPer_52_full',
       'exPer_52_path'].apply(lambda x : x.astype(int).sum())

# replace inf with 0 

df_perMea['13_full'] = round(df_perMea['actual_13_full']/df_perMea['exPer_13_full'])
df_perMea['26_full'] = round(df_perMea['actual_26_full']/df_perMea['exPer_26_full'])
df_perMea['26_path'] = round(df_perMea['actual_26_path']/df_perMea['exPer_26_path'])
df_perMea['26_wrkast'] = round(df_perMea['actual_26_wrkast']/df_perMea['exPer_26_wrkast'])
df_perMea['26_comb'] = round((df_perMea['26_full']) + (df_perMea['26_path']) + (df_perMea['26_wrkast']), ndigits=2) # combined ratio - using DMS weightings 

df_perMea['52_full'] = round(df_perMea['actual_52_full']/df_perMea['exPer_52_full'])
df_perMea['52_path'] = round(df_perMea['actual_52_path']/df_perMea['exPer_52_path'])
df_perMea['52_comb'] = round((df_perMea['52_full'] * 0.25) + (df_perMea['52_path'])) # combined ratio - using DMS weightings 

# replace infinity -> 
df_perMea.replace([np.inf, -np.inf], 0, inplace=True)
df_perMea.columns

Index(['actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path', 'exPer_13_full',
       'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast', 'exPer_52_full',
       'exPer_52_path', '13_full', '26_full', '26_path', '26_wrkast',
       '26_comb', '52_full', '52_path', '52_comb'],
      dtype='object')

In [70]:
df_perMea

actual_13_full  \
Contract_ID Site_Name                         specialist_site_type_code                   
0212790K    AimBig Employment PROSPECT        MUSK                                    0   
0212791K    AimBig Employment MILE END        MENH                                    0   
            AimBig Employment Pty Ltd. SEATON MENH                                    0   
0212793B    AimBig Employment BURLEIGH HEADS  MENH                                    0   
            AimBig Employment NERANG          MENH                                    0   
...                                                                                 ...   
0212902B    AimBig Employment CAMPBELLTOWN    AALL                                    0   
            AimBig Employment INGLEBURN       AALL                                    0   
0212904D    AimBig Employment HURSTVILLE      AALL                                    0   
            AimBig Employment MIRANDA         AALL                                    0   
            AimBig Employment ROCKDALE        AALL                                    0   

                                                                         actual_26_full  \
Contract_ID Site_Name                         specialist_site_type_code                   
0212790K    AimBig Employment PROSPECT        MUSK                                  100   
0212791K    AimBig Employment MILE END        MENH                                   80   
            AimBig Employment Pty Ltd. SEATON MENH                                    0   
0212793B    AimBig Employment BURLEIGH HEADS  MENH                                  120   
            AimBig Employment NERANG          MENH                                    0   
...                                                                                 ...   
0212902B    AimBig Employment CAMPBELLTOWN    AALL                                  160   
            AimBig Employment INGLEBURN       AALL                                    0   
0212904D    AimBig Employment HURSTVILLE      AALL                                  100   
            AimBig Employment MIRANDA         AALL                                    0   
            AimBig Employment ROCKDALE        AALL                                    0   

                                                                         actual_26_path  \
Contract_ID Site_Name                         specialist_site_type_code                   
0212790K    AimBig Employment PROSPECT        MUSK                                    0   
0212791K    AimBig Employment MILE END        MENH                                  180   
            AimBig Employment Pty Ltd. SEATON MENH                                  240   
0212793B    AimBig Employment BURLEIGH HEADS  MENH                                    0   
            AimBig Employment NERANG          MENH                                    0   
...                                                                                 ...   
0212902B    AimBig Employment CAMPBELLTOWN    AALL                                  220   
            AimBig Employment INGLEBURN       AALL                                    0   
0212904D    AimBig Employment HURSTVILLE      AALL                                    0   
            AimBig Employment MIRANDA         AALL                                    0   
            AimBig Employment ROCKDALE        AALL                                    0   

                                                                         actual_26_wrkast  \
Contract_ID Site_Name                         specialist_site_type_code                     
0212790K    AimBig Employment PROSPECT        MUSK                                      0   
0212791K    AimBig Employment MILE END        MENH                                      0   
            AimBig Employment Pty Ltd. SEATON MENH                                      0   
0212793B    AimBig Employment BURLEIGH HEADS  MENH      

### Step 4: Standardise the performance measure scores
Ratio of actual rate pre-standardisation and post-standardisation.

    - scale 0 to 4
    - Using MinMaxScalar

In [57]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,4))
# subset the dataframe, then rejoin -> RESET_INDEX first to transform into dataframe, then rejoin, then groupBy




scaler.fit_transform(df_perMea.loc[:, ~df_perMea.columns.isin(['actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path', 'exPer_13_full',
       'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast', 'exPer_52_full',
       'exPer_52_path'])])

df_perMea


actual_13_full  \
Contract_ID Site_Name                         specialist_site_type_code                   
0212790K    AimBig Employment PROSPECT        MUSK                                    0   
0212791K    AimBig Employment MILE END        MENH                                    0   
            AimBig Employment Pty Ltd. SEATON MENH                                    0   
0212793B    AimBig Employment BURLEIGH HEADS  MENH                                    0   
            AimBig Employment NERANG          MENH                                    0   
...                                                                                 ...   
0212902B    AimBig Employment CAMPBELLTOWN    AALL                                    0   
            AimBig Employment INGLEBURN       AALL                                    0   
0212904D    AimBig Employment HURSTVILLE      AALL                                    0   
            AimBig Employment MIRANDA         AALL                                    0   
            AimBig Employment ROCKDALE        AALL                                    0   

                                                                         actual_26_full  \
Contract_ID Site_Name                         specialist_site_type_code                   
0212790K    AimBig Employment PROSPECT        MUSK                                    5   
0212791K    AimBig Employment MILE END        MENH                                    4   
            AimBig Employment Pty Ltd. SEATON MENH                                    0   
0212793B    AimBig Employment BURLEIGH HEADS  MENH                                    6   
            AimBig Employment NERANG          MENH                                    0   
...                                                                                 ...   
0212902B    AimBig Employment CAMPBELLTOWN    AALL                                    8   
            AimBig Employment INGLEBURN       AALL                                    0   
0212904D    AimBig Employment HURSTVILLE      AALL                                    5   
            AimBig Employment MIRANDA         AALL                                    0   
            AimBig Employment ROCKDALE        AALL                                    0   

                                                                         actual_26_path  \
Contract_ID Site_Name                         specialist_site_type_code                   
0212790K    AimBig Employment PROSPECT        MUSK                                    0   
0212791K    AimBig Employment MILE END        MENH                                    9   
            AimBig Employment Pty Ltd. SEATON MENH                                   12   
0212793B    AimBig Employment BURLEIGH HEADS  MENH                                    0   
            AimBig Employment NERANG          MENH                                    0   
...                                                                                 ...   
0212902B    AimBig Employment CAMPBELLTOWN    AALL                                   11   
            AimBig Employment INGLEBURN       AALL                                    0   
0212904D    AimBig Employment HURSTVILLE      AALL                                    0   
            AimBig Employment MIRANDA         AALL                                    0   
            AimBig Employment ROCKDALE        AALL                                    0   

                                                                         actual_26_wrkast  \
Contract_ID Site_Name                         specialist_site_type_code                     
0212790K    AimBig Employment PROSPECT        MUSK                                      0   
0212791K    AimBig Employment MILE END        MENH                                      0   
            AimBig Employment Pty Ltd. SEATON MENH                                      0   
0212793B    AimBig Employment BURLEIGH HEADS  MENH      

### Step 5: Calculate the Overall Performance Score
- Multiplying the standardised performance scores with the DMS/ESS weighting
TO-DO: multiply with the respective DMS/ESS weight

In [67]:
df_perMea

,Contract_ID,Site_Name,specialist_site_type_code,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path,exPer_13_full,...,52_full,52_path,52_comb,overall_perScore,ORGANISATION_NAME,ORGANISATION_CODE,ESA_NAME,ESA_CODE,ALL_ORG_NAT_AVG,star_rating_perc
0,0212790K,AimBig Employment PROSPECT,MUSK,0,5,0,0,0,0,0,...,NaN,NaN,NaN,NaN,AIMBIG EMPLOYMENT PTY LTD,VVFE,Eastern Adelaide SA,4EAA,73.7,NaN
1,0212790K,AimBig Employment PROSPECT,MUSK,0,5,0,0,0,0,0,...,NaN,NaN,NaN,NaN,AIMBIG EMPLOYMENT PTY LTD,VVFE,Eastern Adelaide SA,4EAA,71.9,NaN
2,0212790K,AimBig Employment PROSPECT,MUSK,0,5,0,0,0,0,0,...,NaN,NaN,NaN,NaN,AIMBIG EMPLOYMENT PTY LTD,VVFE,Eastern Adelaide SA,4EAA,22.8,NaN
3,0212790K,AimBig Employment PROSPECT,MUSK,0,5,0,0,0,0,0,...,NaN,NaN,NaN,NaN,AIMBIG EMPLOYMENT PTY LTD,VVFE,Eastern Adelaide SA,4EAA,19.5,NaN
4,0212790K,AimBig Employment PROSPECT,MUSK,0,5,0,0,0,0,0,...,NaN,NaN,NaN,NaN,AIMBIG EMPLOYMENT PTY LTD,VVFE,Eastern Adelaide SA,4EAA,24.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,0212904D,AimBig Employment ROCKDALE,AALL,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,AIMBIG EMPLOYMENT PTY LTD,VVFE,St George-Sutherland,4SGS,0.0,NaN
3036,0212904D,AimBig Employment ROCKDALE,AALL,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,AIMBIG EMPLOYMENT PTY LTD,VVFE,St George-Sutherland,4SGS,22.4,NaN
3037,0212904D,AimBig Employment ROCKDALE,AALL,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,AIMBIG EMPLOYMENT PTY LTD,VVFE,St George-Sutherland,4SGS,1.3,NaN
3038,0212904D,AimBig Employment ROCKDALE,AALL,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,AIMBIG EMPLOYMENT PTY LTD,VVFE,St George-Sutherland,4SGS,21.5,NaN


In [58]:
df_perMea['13_full'] = df_perMea['13_full'] * 0.20
df_perMea['26_full'] = df_perMea['26_full'] * 0.40
df_perMea['52_comb'] = df_perMea['52_comb'] * 0.25

df_perMea['overall_perScore'] = df_perMea['13_full'] + df_perMea['26_full'] + df_perMea['52_comb']

### Step 6: Calculate the Star Rating Percentage
- Calculated using the formula:
str% = (performance score for contract X) - (National Average contract perf score)/(National Average contract perf score) * 100 


In [59]:
# bringing in the sub368 file to read in the national averages
df_avg = pd.read_csv("C:/Users/AManalo/star_ratings_new/Sub368_Data_Sep2022.csv")
df_avg.columns

Index(['REPORT_DATE', 'EOM_SEQ', 'EOW_SEQ', 'DES_PROGRAM', 'ORGANISATION_NAME',
       'ORGANISATION_CODE', 'SUMMARY_LINE_TYPE', 'LMR_NAME', 'LMR_CODE',
       'ESA_NAME', 'ESA_CODE', 'CONTRACT_ID', 'SITE_NAME', 'SITE_CODE',
       'LOCALITY', 'SITE_TYPE', 'SPECIALTY_CD', 'START_DATE', 'END_DATE',
       'PROV_EOM_SEQ', 'PROV_EOW_SEQ', 'CONTRACT_START_GROUP',
       'PERFORMANCE_MEASURES', 'RATE', 'NUMERATOR', 'DENOMINATOR',
       'OWN_ORG_ESA_AVG', 'OWN_ORG_LMR_AVG', 'OWN_ORG_NAT_AVG',
       'ALL_ORG_SAME_SPEC_ESA_AVG', 'ALL_ORG_SAME_SPEC_LMR_AVG',
       'ALL_ORG_SAME_SPEC_NAT_AVG', 'ALL_ORG_ESA_AVG', 'ALL_ORG_LMR_AVG',
       'ALL_ORG_NAT_AVG', 'NAT_AVG_ALL_SPEC_JULY_2018'],
      dtype='object')

Joining the sub368 file with the current dataframe to get the national average

str% = (performance score for contract X) - (National Average contract perf score)/(National Average contract perf score) * 100 

In [60]:
df_perMea = df_perMea.reset_index()

In [61]:
df_avg

,REPORT_DATE,EOM_SEQ,EOW_SEQ,DES_PROGRAM,ORGANISATION_NAME,ORGANISATION_CODE,SUMMARY_LINE_TYPE,LMR_NAME,LMR_CODE,ESA_NAME,...,OWN_ORG_ESA_AVG,OWN_ORG_LMR_AVG,OWN_ORG_NAT_AVG,ALL_ORG_SAME_SPEC_ESA_AVG,ALL_ORG_SAME_SPEC_LMR_AVG,ALL_ORG_SAME_SPEC_NAT_AVG,ALL_ORG_ESA_AVG,ALL_ORG_LMR_AVG,ALL_ORG_NAT_AVG,NAT_AVG_ALL_SPEC_JULY_2018
0,30/09/2022,0.0,5.0,DES - DMS,AIMBIG EMPLOYMENT PTY LTD,VVFE,OUTLET,Adelaide,4ADE,Eastern Adelaide SA,...,0.0,100.0,64.7,0.0,0.0,42.9,70.0,68.8,73.7,42.9
1,30/09/2022,0.0,5.0,DES - DMS,AIMBIG EMPLOYMENT PTY LTD,VVFE,OUTLET,Adelaide,4ADE,Eastern Adelaide SA,...,0.0,100.0,64.7,0.0,0.0,42.9,50.0,66.3,71.9,42.9
2,30/09/2022,0.0,5.0,DES - DMS,AIMBIG EMPLOYMENT PTY LTD,VVFE,OUTLET,Adelaide,4ADE,Eastern Adelaide SA,...,19.5,18.1,19.8,15.9,15.1,20.5,18.2,17.6,19.5,20.7
3,30/09/2022,0.0,5.0,DES - DMS,AIMBIG EMPLOYMENT PTY LTD,VVFE,OUTLET,Adelaide,4ADE,Eastern Adelaide SA,...,0.0,0.6,0.8,0.0,1.5,0.7,2.9,1.9,1.2,0.7
4,30/09/2022,0.0,5.0,DES - DMS,AIMBIG EMPLOYMENT PTY LTD,VVFE,OUTLET,Adelaide,4ADE,Eastern Adelaide SA,...,19.5,18.1,19.8,15.9,15.1,20.5,18.2,17.6,19.5,20.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
df_avg = df_avg.rename(columns={'CONTRACT_ID' : 'Contract_ID', 'SITE_NAME' : 'Site_Name', 'SPECIALTY_CD' : 'specialist_site_type_code'})

df_avg = df_avg[['ORGANISATION_NAME','ORGANISATION_CODE', 'ESA_NAME', 'ESA_CODE', 'Contract_ID', 'Site_Name', 'specialist_site_type_code', 'ALL_ORG_NAT_AVG']]

# join df_avg to df_perMea

df_perMea = df_perMea.merge(df_avg, on=['Contract_ID', 'Site_Name', 'specialist_site_type_code'], how='inner')



In [63]:
# calculating the star rating percentage
df_perMea['star_rating_perc'] = (df_perMea['overall_perScore'] - df_perMea['ALL_ORG_NAT_AVG'])/ (df_perMea['ALL_ORG_NAT_AVG']) * 100

In [66]:
df_perMea.to_csv('C:/Users/AManalo/star_ratings_new/star_ratings.csv')